In [3]:
# Libraries needed for WebCam Face_Detection.
import cv2
import os
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import InceptionResnetV1,MTCNN
import sqlite3
from scipy.spatial.distance import euclidean

In [4]:
os.getcwd()

'C:\\Users\\yeswa'

In [5]:
os.chdir("C:\\Users\\yeswa\\OneDrive\\Pictures\\Face_Recognization_Photos\\Yeshwnath")

In [6]:
#below variables are FaceNet related-> FaceNet creates Embedding Vectors of each face detected by MTCNN model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')# This makes the enviroment use GPU if exist other wise CPU because of performance.
facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)# creating object for FaceNet Model.
mtcnn = MTCNN(keep_all=True,device=device)# Creating object for MTCNN model.

In [7]:
folder_path = "C:\\Users\\yeswa\\OneDrive\\Pictures\\Face_Recognization_Photos\\Yeshwnath"
def get_known_embedding(path):
    embedding_dict = {}
    for folder_path in os.listdir(path):
        images_path = os.path.join(path,folder_path)
        bgr_images = cv2.imread(images_path)
        if bgr_images is None:
            print("error to fetch frames",{images_path})
            continue
        rgb_faces = cv2.cvtColor(bgr_images,cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(rgb_faces)
        detect_faces = mtcnn(img_pil)
        if detect_faces is None:
            print("No Face is Detected:",{images_path})
            continue
        if detect_faces.ndim==3:
            batch_tensor = detect_faces.unsqueeze(0).to(device)
        else:
            batch_tensor = detect_faces.to(device)
        embedding = facenet(batch_tensor).detach().cpu().squeeze(0)
    return embedding

In [8]:
known_faces = get_known_embedding(folder_path)
known_faces

tensor([ 1.1934e-02, -9.0208e-04, -4.6704e-02,  3.0958e-02,  4.1157e-03,
         3.7655e-02,  5.6587e-02,  5.4107e-02, -1.4423e-02,  1.2453e-02,
        -1.9306e-02, -6.4029e-03, -6.1888e-02, -1.2130e-03, -8.9265e-03,
         1.5313e-02,  4.5176e-02,  5.0795e-02,  4.5453e-03,  4.0759e-03,
        -5.6188e-03, -5.2012e-03,  9.4744e-03, -3.1029e-02,  6.1819e-03,
         7.9293e-03,  2.3836e-02,  1.0526e-02,  2.3628e-02, -5.7807e-02,
        -1.8780e-02,  3.3934e-02, -5.5936e-02,  6.9764e-02, -3.6331e-02,
         3.0756e-02, -5.8294e-02, -2.2129e-02, -2.2624e-02,  3.1380e-02,
         6.2032e-02, -3.7578e-02,  4.8310e-03,  2.3327e-02, -8.5855e-03,
        -1.8994e-02, -4.9212e-02,  2.7319e-02, -3.4050e-02, -8.7336e-03,
        -3.5678e-03, -1.5571e-02,  1.0256e-01,  4.9071e-02, -4.2501e-02,
        -6.8495e-03, -1.6458e-02,  7.1385e-03,  4.5265e-02, -3.9048e-02,
        -2.0623e-02,  2.5057e-02,  7.6368e-03,  4.2826e-02, -3.6992e-03,
        -2.1826e-02, -4.2928e-02, -2.2712e-02, -1.9

In [9]:
# below code is read frames of webcam
cap = cv2.VideoCapture(0) # initate the webcam
while True: # if flow is true then go inside the loop
    ret,frame = cap.read() # ret->True/Fasle of VideoCapturing if capturing True not capturing False
    if not ret: #condition on frames are getting or not.
        break # if condition true then break flow here itself.
    boxes,prob = mtcnn.detect(frame)# Detects faces from CC Camera and shows green box around face->box,prob->indicates accuracy of face >=95 then it is face.
    if boxes is not None:
        for box in boxes:
            x1,y1,x2,y2 = [int(coord) for coord in box]
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            # Above loop and two lines are going to display green line around the face lively.
            face = frame[y1:y2,x1:x2]# This variable consist of that face numerical vector
            face_tensor = mtcnn(frame)
            if face_tensor is None:
                continue
            if len(face_tensor.shape) == 3:
                batch_face = face_tensor.unsqueeze(0).to(device)
            else:
                batch_face = face_tensor.to(device)
            live_embeddings = facenet(batch_face).detach().cpu().squeeze(0)
            distance = euclidean(known_faces.numpy(),live_embeddings.numpy())
            if distance<0.8:
                print("Known person")
            elif distance>=1.0:
                print("unknown person")
    cv2.imshow("Frame",frame)# Display the image from vectors.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Known person
Known person
Known person
Known person
Known person
Known person
Known person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
unknown person
Known person
